<a href="https://colab.research.google.com/github/antahiap/dsr-tut-productive-ml/blob/master/notebooks/dl_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [1]:
# !python -m pip install --upgrade pip
%pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


**A KerasTuner model-building function**

In [3]:
!python -V

Python 3.10.12


In [2]:
from tensorflow import keras

# Senario 1
Defining a function

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

In [4]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

<ipython-input-4-643dcc05e516>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [6]:
tuner = kt.BayesianOptimization(
    build_model, #hypermodel
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [8]:
# load data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#  reshape data
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

# split data for validation
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

# make hyper parameter study
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=4,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.9581499993801117

Best val_accuracy So Far: 0.9633499979972839
Total elapsed time: 00h 01m 28s


**Querying the best hyperparameter configurations**

In [15]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

# Print the best hyperparameters in a readable format
for bi in best_hps:
  for param, value in bi.values.items():
      print(f"{param}: {value}")

units: 64
optimizer: rmsprop
units: 48
optimizer: rmsprop
units: 48
optimizer: adam
units: 32
optimizer: adam


# Architecture hyperparam

In [26]:
def build_model_2(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)

    num_layers = hp.Int(name="num_layers", min_value=1, max_value=3, step=1)

    units = [layers.Dense(units, activation="relu") for layer in range(num_layers)]

    model = keras.Sequential(
        units+
        [layers.Dense(10, activation="softmax")]
    )
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

In [27]:
tuner_2 = kt.BayesianOptimization(
    build_model_2, #hypermodel
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [28]:
tuner_2.search_space_summary()

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [29]:
# load data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#  reshape data
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

# split data for validation
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

# make hyper parameter study
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner_2.search(
    x_train, y_train,
    batch_size=128,
    epochs=4,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 5 Complete [00h 00m 13s]
val_accuracy: 0.9546999931335449

Best val_accuracy So Far: 0.9584999978542328
Total elapsed time: 00h 01m 19s
